In [26]:
import os
from nltk import sent_tokenize
import re

In [27]:
with open('AliceInWonderland.txt', 'r', encoding = 'utf-8') as f:
    data = f.read()

Выбираем кусок, с которым дальше будем работать.

In [28]:
data_ideal = data[1522:4981]

Пишем регулярку, которая должна поделить текст на предложения. Позже полученное доведем до идеала вручную.

In [29]:
regex = r'([.|!|?|...|\n])\s+'

In [30]:
my_split1 = re.split(regex, data_ideal)

Моя регулярка удаляла символы, по которым делила текст (и писала их в отдельный элемент массива), поэтому здесь я соединяю элементы полученного массива, чтобы точки стояли после своих предложений.

In [31]:
my_split = []
for i in range(0, len(my_split1)-1):
    if i%2 == 0:
        my_split1[i] = my_split1[i]+my_split1[i+1]
for z in range(0, len(my_split1)):
    if z%2 == 0:
        my_split.append(my_split1[z])

In [32]:
my_split

[' Но стоило мне заикнуться об этом своем желании, как все начинали на меня страшно кричать, чтобы я не смел.',
 'И я не посмел!',
 'Все горе в том, что книжка эта была написана в Англии сто лет тому назад и за это время успела так прославиться, что и у нас все – хотя бы понаслышке – знают про Алису и привыкли к скучноватому названию «Приключения Алисы в Стране Чудес».',
 'Это называется литературной традицией, и тут, как говорится, ничего не попишешь.',
 'Хотя название «Алиска в Расчудесии» гораздо больше похоже на настоящее, английское название этой сказки; но если бы я ее так назвал, люди подумали бы, что это совершенно другая книжка, а не та, знаменитая…\n',
 'А знаменита «Алиса» действительно сверх всякой меры.',
 'В особенности в тех странах, где говорят по-английски.',
 'Там ее знает каждый и любят все.',
 'И самое интересное, что, хотя эта сказка для детей, пожалуй, больше детей любят ее взрослые, а больше всех – самые взрослые из взрослых – ученые!',
 'Да, сразу видно, что это

Количество предложений, которое получила регулярка:

In [33]:
len(my_split)

42

Сохраним файл, чтобы вручную его поправить.

In [34]:
with open('ideal1.txt', 'w', encoding = 'utf-8') as t:
    for i in my_split:
        t.write(i+'\n')

Ниже попытка обойтись без сплита, который удяляет пунктуацию :) Однако провальная судя по длине

In [35]:
ideal2 = re.findall(r'[А-ЯЁA-Z].*[.!?…\n]\s', data_ideal)

In [36]:
len(ideal2)

22

Откроем поправленный файл, посчитаем количество строк (предложений) в нем и добавим в новый идеальный массив.

In [37]:
with open('ideal1.txt', 'r', encoding = 'utf-8') as i:
    i = i.readlines()
    gold_len = len(i)
    gold = []
    for z in i:
        z = z.strip('\n')
        gold.append(z)

In [38]:
gold

['Но стоило мне заикнуться об этом своем желании, как все начинали на меня страшно кричать, чтобы я не смел.',
 'И я не посмел!',
 'Все горе в том, что книжка эта была написана в Англии сто лет тому назад и за это время успела так прославиться, что и у нас все – хотя бы понаслышке – знают про Алису и привыкли к скучноватому названию «Приключения Алисы в Стране Чудес».',
 'Это называется литературной традицией, и тут, как говорится, ничего не попишешь.',
 'Хотя название «Алиска в Расчудесии» гораздо больше похоже на настоящее, английское название этой сказки; но если бы я ее так назвал, люди подумали бы, что это совершенно другая книжка, а не та, знаменитая…',
 'А знаменита «Алиса» действительно сверх всякой меры.',
 'В особенности в тех странах, где говорят по-английски.',
 'Там ее знает каждый и любят все.',
 'И самое интересное, что, хотя эта сказка для детей, пожалуй, больше детей любят ее взрослые, а больше всех – самые взрослые из взрослых – ученые!',
 'Да, сразу видно, что это оч

In [39]:
gold_len

39

gold_len - идеальная длина, которую я получила, подправив руками то, что дала регулярка. Дело в том, что регулярка засекла случаи типа "Нужно – и это совершенно обязательно! – иметь чувство юмора, потому что это одна из самых веселых книжек на свете." и после "обязательно!" отделила новое предложение. Таких случаев было три, текст не самый обычный :)

Посчитаем также совпадающие элементы и посмотрим, что говорит нам эта метрика.

In [40]:
len(set(gold) & set(my_split)) / len(set(gold) | set(my_split))

0.6122448979591837

Обучим PunktSentenceTokenizer на всем тексте.

In [41]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [42]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train('\n'.join(data))
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

In [43]:
print(tokenizer._params.abbrev_types)

set()


Посмотрим, как этот метод разбивает наш кусок на предложения.

In [44]:
tok_data = tokenizer.tokenize(data_ideal)

In [45]:
tok_data

[' Но стоило мне заикнуться об этом своем желании, как все начинали на меня страшно кричать, чтобы я не смел.',
 'И я не посмел!',
 'Все горе в том, что книжка эта была написана в Англии сто лет тому назад и за это время успела так прославиться, что и у нас все – хотя бы понаслышке – знают про Алису и привыкли к скучноватому названию «Приключения Алисы в Стране Чудес».',
 'Это называется литературной традицией, и тут, как говорится, ничего не попишешь.',
 'Хотя название «Алиска в Расчудесии» гораздо больше похоже на настоящее, английское название этой сказки; но если бы я ее так назвал, люди подумали бы, что это совершенно другая книжка, а не та, знаменитая…\n\n   \nА знаменита «Алиса» действительно сверх всякой меры.',
 'В особенности в тех странах, где говорят по-английски.',
 'Там ее знает каждый и любят все.',
 'И самое интересное, что, хотя эта сказка для детей, пожалуй, больше детей любят ее взрослые, а больше всех – самые взрослые из взрослых – ученые!',
 'Да, сразу видно, что э

In [46]:
len(tok_data)

40

Этот метод, как и регулярка, увидел "Нужно – и это совершенно обязательно! – иметь чувство юмора, потому что это одна из самых веселых книжек на свете." и после "обязательно!" отделил новое предложение. Однако он не заметил больше одного переноса строки, поэтому метрика со сравнением длин нас здесь обманывает. 

In [47]:
len(set(tok_data) & set(gold)) / len(set(tok_data) | set(gold))

0.6041666666666666

Следующая метрика показывает на самом деле, насколько одинаково разбивает на предлоежения sent_tokenize и tokenizer_tokenize. Видимо, одинаково :)

In [48]:
tp = 0
fp = 0
fn = 0

for sent in tok_data:
    if len(sent_tokenize(sent)) == 1:
        tp += 1
    else:
        fp += 1

for i in range(len(tok_data)-1):
    sent1, sent2 = tok_data[i], tok_data[i+1]
    sent = ' '.join([sent1, sent2])
    if len(sent_tokenize(sent)) == 2:
        tp += 1
    else:
        fn += 1

precision = (tp/(tp+fp))
recall = (tp/(tp+fn))
f1 = 2*(precision*recall)/(precision+recall)
print('Precision - ', precision)
print('Recall - ', recall)
print('F1 - ', f1)

Precision -  1.0
Recall -  1.0
F1 -  1.0
